![](https://cdn-images-1.medium.com/max/1600/1*jX6Gwn1rt4da7e-yUj84IQ.png)

### 請先在terminal執行
`pip install unidecode`

`pip install psutil`

- 如果psutil失敗請把下面這個 cell，comment起來

Example: 
```
def cpuStats():
    """ @author: RDizzl3 @address: https://www.kaggle.com/rdizzl3"""
    print("########## CPU STATS ############")
    #pid = os.getpid()
    #print(pid)
    #py = psutil.Process(pid)
    #memoryUse = py.memory_info()[0] / 2. ** 30
    #print('memory GB:', memoryUse)
    #print("########## CPU STATS ############")
```


In [ ]:
import sys, os, psutil


def cpuStats():
    """ @author: RDizzl3 @address: https://www.kaggle.com/rdizzl3"""
    print("########## CPU STATS ############")
    pid = os.getpid()
    print(pid)
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2. ** 30
    print('memory GB:', memoryUse)
    print("########## CPU STATS ############")

In [ ]:
import gc
import time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack, vstack
from collections import Counter
from sklearn.svm import LinearSVR
from sklearn.linear_model import Ridge, SGDRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.preprocessing import LabelBinarizer, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from multiprocessing import Process, Pool
import functools
from scipy.special import erfinv
from scipy.sparse.linalg import norm
import re
import unidecode
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer
from sklearn.decomposition import TruncatedSVD, NMF
import warnings
import math

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
PATH = 'data/'
train = pd.read_table(PATH + 'train.tsv', engine='c')
test = pd.read_table(PATH + 'test.tsv', engine='c')

train = train.loc[train.price > 0]
train.reset_index(drop=True, inplace=True)
y = np.log1p(train["price"].values)

## 幫助函數，請忽視下面這個Cell



In [ ]:
Hash_binary = True

def handle_missing_inplace(dataset):
    dataset['category_name'].fillna(value='missing', inplace=True)
    dataset['brand_name'].fillna(value='missing', inplace=True)
    dataset['item_description'].fillna(value='No description yet', inplace=True)

    
def preprocess(text): # 這是之前的作業的簡易版
    non_alphanums = re.compile(u'[^A-Za-z0-9]+')
    # regex for short = re 請參考 http://ccckmit.wikidot.com/regularexpression 
    
    text = unidecode.unidecode(text)
    text = str(text).lower()
    return u" ".join(
        [x for x in [y for y in non_alphanums.sub(' ', text).strip().split(" ")]])
        # strip split 請參考 http://ericbbs.blogspot.tw/2009/07/python-strip-split.html
        # [x for x in.....] 這文言文：是 list comprehension

    
### 以下是multithread ，請自行忽略 超過本課程討論範圍  

def multi_hash(data=None, hash_vec=None, n_jobs=4):

    p = Pool(n_jobs)
    csr_parts = p.map(hash_vec.fit_transform, np.array_split(data, n_jobs))
    p.close()
    p.join()
    return vstack(csr_parts).tocsr

def multi_apply(df=None, feat_list=None, func=None, axis=0, raw=True, n_jobs=4):

    p = Pool(n_jobs)
    f_ = p.map(functools.partial(apply_func, func=func, axis=axis, raw=raw),
               np.array_split(df[feat_list], n_jobs))
    f_ = pd.concat(f_, axis=0, ignore_index=True)
    p.close()
    p.join()
    return f_.values

def apply_func_series(data=None, func=None):

    return data.apply(func)

def multi_apply_series(df=None, feature=None, func=None, n_jobs=4):

    p = Pool(n_jobs)
    f_ = p.map(functools.partial(apply_func_series, func=func),
               np.array_split(df[feature], n_jobs))
    f_ = pd.concat(f_, axis=0, ignore_index=True)
    p.close()
    p.join()
    return f_.values
    

def apply_func(data=None, func=None, axis=0, raw=True):

    return data.apply(func, axis=axis, raw=raw)


def preprocess_text_features(df):
    """
      Utility function to apply text pre-processing by Kueipo to name, brand and description
      but in parallel
      """
    df["item_description"] = multi_apply_series(df=df[["item_description"]],
                                                feature="item_description",
                                                func=preprocess,
                                                n_jobs=4)
    df["name"] = multi_apply_series(df=df[["name"]],
                                    feature="name",
                                    func=preprocess,
                                    n_jobs=4)
    
def get_hashing_features(df, Hash_binary, start_time):
    # df = pd.concat([train, test])
    dim = 20
    ha = HashingVectorizer(
        n_features=2 ** dim,
        ngram_range=(1, 2),
        norm=None,
        alternate_sign=False,
        binary=Hash_binary
        # stop_words='english'
    )

    X_name = ha.fit_transform(df['name'])
    cpuStats()
    X_name += ha.fit_transform(df['item_description'])
    cpuStats()
    
    print('[{}] Finished hashing'.format(time.time() - start_time))
    return X_name

In [ ]:
start_time = time.time()
handle_missing_inplace(train) # 處理 NaN 
preprocess_text_features(df=train) # 文字預處理，前一份exercise

csr_trn = get_hashing_features(train, Hash_binary, start_time) # Hash Trick

print('Shape of this Sparse Matrix: {}'.format(csr_trn.shape[1]))

Shape of this Sparse Matrix: 1048576


### 特徵值有一百零四萬多，請練習使用Lasso選特徵
- 請參看[SGDRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor) attributes
`coef_`

In [ ]:
# Apply L1 feature selection
print('[{}] Starting SGD l1 selection'.format(time.time() - start_time))
sgd_l1 = SGDRegressor(max_iter=30, penalty="l1", random_state=1, alpha=1e-6)
sgd_l1.fit(csr_trn, y)

[826.8082292079926] Starting SGD l1 selection


SGDRegressor(alpha=1e-06, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=30, n_iter=None, penalty='l1',
       power_t=0.25, random_state=1, shuffle=True, tol=None, verbose=0,
       warm_start=False)

# 練習時間：
- Hint 1:
    - Lasso 會陽痿一堆欄位係數
- Hint 2:
    - 您要做的事情就是挑選係數範圍，並排除超小係數，範圍自訂
    
請看被L1處理後的係數

In [ ]:
sgd_l1.coef_

array([0.05418402, 0.00060792, 0.        , ..., 0.00335522, 0.        ,
       0.02573101])

#### Excercise 1:
      - 請檢視最大值，最小值

In [ ]:
# 發現最小值特性.....

#### Excercise 2:
- 請挑選係數範圍
- Hint:
    1. 請記得用絕對值np.abs(.....)，如果有做 Excercise 1，會知道為什麼
    2. 請轉成 np.array()

In [ ]:
print('[{}] Finished SGD l1 selection'.format(time.time() - start_time))

good_feats = # your code goes here, YES 就是只有一行

print("Features reduced from %10d to %10d" % (csr_trn.shape[1], int(good_feats.sum())))